<a href="https://colab.research.google.com/github/ALIN-BIRD/Deep-Learning-Active-Noise-Cancellation-ANC-for-CNC-Chatter/blob/main/Chatter_noise_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os

print("📂 Please click the button below to upload your file:")
uploaded = files.upload()

# Check what was uploaded and rename if necessary
for filename in uploaded.keys():
    print(f"✅ User uploaded: {filename}")

    # If the file is named slightly differently (e.g., 'Chatter_noise (1).csv'), fix it
    if filename != 'Chatter_noise.csv':
        os.rename(filename, 'Chatter_noise.csv')
        print(f"🔄 Renamed '{filename}' to 'Chatter_noise.csv' for you.")

📂 Please click the button below to upload your file:


Saving Chatter_noise.csv to Chatter_noise.csv
✅ User uploaded: Chatter_noise.csv


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import os

# ==========================================
# 0. GPU SETUP
# ==========================================
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 High-Performance Mode: {device}")

# ==========================================
# 1. LOAD DATA & CONFIG
# ==========================================
SAMPLE_RATE = 8000
# HYPERPARAMETER 1: Larger Window for better context
WINDOW_SIZE = 400

file_path = '/content/Chatter_noise.csv'
if not os.path.exists(file_path):
    print("❌ ERROR: Please upload 'Chatter_noise.csv' again!")
    raise SystemExit

df = pd.read_csv(file_path, header=None)
raw_data = df.iloc[:, -1].values
max_val = np.max(np.abs(raw_data))
if max_val == 0: max_val = 1
noise_source = raw_data / max_val # Normalize -1 to 1

# ==========================================
# 2. VECTORIZED DATA PREP
# ==========================================
print("⚡ Processing Data...")
data_tensor = torch.FloatTensor(noise_source).to(device)

# Instant Windowing
X_tensor = data_tensor.unfold(0, WINDOW_SIZE, 1)
y_tensor = data_tensor[WINDOW_SIZE:].view(-1, 1)
X_tensor = X_tensor[:y_tensor.shape[0]]

# HYPERPARAMETER 2: Larger Batch Size for stability
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=512, shuffle=True)

# ==========================================
# 3. THE "DEEP" BRAIN (3 Layers, 256 Units)
# ==========================================
class CNC_Turbo(nn.Module):
    def __init__(self):
        super(CNC_Turbo, self).__init__()
        # HYPERPARAMETER 3 & 4: Deeper (3 layers) and Wider (256 units)
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=3, batch_first=True, dropout=0.2)

        # Extra dense layers to refine the output
        self.fc1 = nn.Linear(256, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.unsqueeze(-1)
        lstm_out, _ = self.lstm(x)
        last_step = lstm_out[:, -1, :]
        x = self.relu(self.fc1(last_step))
        return self.fc2(x)

model = CNC_Turbo().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# HYPERPARAMETER 5: Learning Rate Scheduler
# If loss stops dropping for 10 epochs, cut LR in half. This fixes "jitter".
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

# ==========================================
# 4. TRAINING LOOP (Long Run)
# ==========================================
epochs = 300 # HYPERPARAMETER 6: Long training for perfect phase alignment
print(f"🧠 Training for {epochs} epochs with Scheduler...")

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)

    # Update the scheduler (it will print if it lowers the LR)
    scheduler.step(avg_loss)

    if (epoch + 1) % 20 == 0:
        print(f"   Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.7f}")

# ==========================================
# 5. RESULTS & BATCHED PREDICTION
# ==========================================
print("\n📊 Generating Final Graph...")
model.eval()
all_preds = []
BATCH_SIZE = 2000

with torch.no_grad():
    for i in range(0, len(X_tensor), BATCH_SIZE):
        batch_in = X_tensor[i:i+BATCH_SIZE]
        batch_out = model(batch_in)
        all_preds.append(batch_out.cpu().numpy())

prediction = np.concatenate(all_preds, axis=0)
actual = y_tensor.cpu().numpy()
anti_noise = -prediction
residual = actual + anti_noise

# Calculate dB
p_in = np.mean(actual**2)
p_out = np.mean(residual**2)
db_red = 10 * np.log10(p_in/p_out) if p_out > 0 else 0

print(f"\n🎉 FINAL RESULT: {db_red:.2f} dB Reduction")

plt.figure(figsize=(12, 10))
plt.subplot(3,1,1); plt.plot(actual, 'k'); plt.title("1. Original Chatter")
plt.subplot(3,1,2); plt.plot(anti_noise, 'r'); plt.title("2. AI Prediction")
plt.subplot(3,1,3); plt.plot(residual, 'g'); plt.title(f"3. Result ({db_red:.2f} dB)")
plt.tight_layout()
plt.show()

write("/content/cnc_fixed_turbo.wav", SAMPLE_RATE, np.int16(residual * 32767))